## 데이터 전처리

### 실습 데이터셋 안내

- 2021년 1~12월에 전세계 165여 개 국가와 수산물 수출입 내역입니다.<br><br>

- 변수(열)에 대한 간단한 설명입니다.
  - stdYymm: 기준년월(yyyymm)
  - nationCode: 국가코드
  - mprcExipitmCode: 수산물수출입품목코드(10자리 숫자)
  - imxprtSeCode: 수출입구분코드(E, I)
  - nationNm: 국가명
  - ecoblNm: 경제권명
  - mprcExipitmNm: 수산물수출입품목명
  - imxprtSeNm: 수출입구분명(수출, 수입)
  - imxprtWt: 수출입중량(kg)
  - imxprtDollarAmount: 수출입금액($)<br><br>

- 실습 데이터인 csv 파일을 아래 링크로 제공합니다.
  - https://bit.ly/Seafood_Trade_2021_CP949
  - 구분자는 콤마(,)이고 인코딩방식은 CP949입니다.

### 관련 라이브러리 호출

In [1]:
# 관련 라이브러리를 호출합니다.
import requests
import chardet
import numpy as np
import pandas as pd
import os
import joblib

### 인코딩 방식 확인

In [2]:
# 온라인으로 공유 중인 csv 파일 주소를 변수에 할당합니다.
url = 'https://bit.ly/Seafood_Trade_2021_CP949'

In [3]:
# 링크에 있는 텍스트를 내려받습니다.(HTTP Requests)
# [힌트] requests.get() 함수를 사용하세요.
res = requests.get(url = url)

In [4]:
# res에서 바이너리 텍스트의 일부를 출력합니다.
# [힌트] content 속성을 사용하세요.
res.content[:200]

b'stdYymm,nationCode,mprcExipitmCode,imxprtSeCode,nationNm,ecoblNm,mprcExipitmNm,imxprtSeNm,imxprtWt,imxprtDollarAmount\n202101,TW,2102203090,I,\xb4\xeb\xb8\xb8,\xbe\xc6\xbd\xc3\xbe\xc6.\xc5\xc2\xc6\xf2\xbe\xe7 \xb0\xe6\xc1\xa6\xc7\xf9\xb7\xc2\xc3\xbc,\xc5\xac\xb7\xce\xb7\xbc\xb6\xf3 \xc8\xbf\xb8\xf0,\xbc\xf6\xc0\xd4,500,13456\n'

In [5]:
# 바이너리 텍스트의 인코딩 방식을 확인합니다.
# [힌트] chardet.detect() 함수를 사용하세요.
chardet.detect(res.content)

{'encoding': 'EUC-KR', 'confidence': 0.99, 'language': 'Korean'}

### 실습 데이터셋 준비

In [6]:
# 링크를 읽고 데이터프레임 sfd를 생성합니다.
# [힌트] pd.read_csv() 함수를 사용하세요.
# [참고] EUC-KR은 CP949의 부분집합입니다.
sfd = pd.read_csv(filepath_or_buffer = url, encoding = 'CP949')

In [7]:
# sfd의 정보를 확인합니다.
# [힌트] info() 함수를 사용하세요.
sfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31522 entries, 0 to 31521
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   stdYymm             31522 non-null  int64  
 1   nationCode          31499 non-null  object 
 2   mprcExipitmCode     31522 non-null  int64  
 3   imxprtSeCode        31522 non-null  object 
 4   nationNm            31522 non-null  object 
 5   ecoblNm             26878 non-null  object 
 6   mprcExipitmNm       28824 non-null  object 
 7   imxprtSeNm          31522 non-null  object 
 8   imxprtWt            31522 non-null  float64
 9   imxprtDollarAmount  31522 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 2.4+ MB


In [8]:
# sfd를 출력합니다.
sfd

,stdYymm,nationCode,mprcExipitmCode,imxprtSeCode,nationNm,ecoblNm,mprcExipitmNm,imxprtSeNm,imxprtWt,imxprtDollarAmount
0,202101,TW,2102203090,I,대만,아시아.태평양 경제협력체,클로렐라 효모,수입,500.00,13456
1,202101,JP,2102203090,I,일본,아시아.태평양 경제협력체/경제협력개발기구,클로렐라 효모,수입,84.00,1326
2,202101,US,2102203090,I,미국,아시아.태평양 경제협력체/경제협력개발기구/북미자유무역연합,클로렐라 효모,수입,4.84,193
3,202101,CA,2102204010,I,캐나다,아시아.태평양 경제협력체/경제협력개발기구/북미자유무역연합,스리루리나 효모,수입,1505.78,33789
4,202101,JP,2102204010,I,일본,아시아.태평양 경제협력체/경제협력개발기구,스리루리나 효모,수입,665.28,17319
...,...,...,...,...,...,...,...,...,...,...
31517,202112,DE,301999070,I,독일,유럽연합/경제협력개발기구,미꾸라지,수입,1.00,242
31518,202112,US,301999070,E,미국,아시아.태평양 경제협력체/경제협력개발기구/북미자유무역연합,미꾸라지,수출,371.00,6209
31519,202112,CN,301999070,I,중국,아시아.태평양 경제협력체,미꾸라지,수입,643363.00,2593251
31520,202112,JP,302896000,I,일본,아시아.태평양 경제협력체/경제협력개발기구,돔,수입,52.80,2249


In [9]:
# sfd의 처음 5행을 출력합니다.
# [힌트] head() 함수를 사용하세요.
sfd.head()

,stdYymm,nationCode,mprcExipitmCode,imxprtSeCode,nationNm,ecoblNm,mprcExipitmNm,imxprtSeNm,imxprtWt,imxprtDollarAmount
0,202101,TW,2102203090,I,대만,아시아.태평양 경제협력체,클로렐라 효모,수입,500.00,13456
1,202101,JP,2102203090,I,일본,아시아.태평양 경제협력체/경제협력개발기구,클로렐라 효모,수입,84.00,1326
2,202101,US,2102203090,I,미국,아시아.태평양 경제협력체/경제협력개발기구/북미자유무역연합,클로렐라 효모,수입,4.84,193
3,202101,CA,2102204010,I,캐나다,아시아.태평양 경제협력체/경제협력개발기구/북미자유무역연합,스리루리나 효모,수입,1505.78,33789
4,202101,JP,2102204010,I,일본,아시아.태평양 경제협력체/경제협력개발기구,스리루리나 효모,수입,665.28,17319


In [10]:
# sfd의 마지막 5행을 출력합니다.
# [힌트] tail() 함수를 사용하세요.
sfd.tail()

,stdYymm,nationCode,mprcExipitmCode,imxprtSeCode,nationNm,ecoblNm,mprcExipitmNm,imxprtSeNm,imxprtWt,imxprtDollarAmount
31517,202112,DE,301999070,I,독일,유럽연합/경제협력개발기구,미꾸라지,수입,1.0,242
31518,202112,US,301999070,E,미국,아시아.태평양 경제협력체/경제협력개발기구/북미자유무역연합,미꾸라지,수출,371.0,6209
31519,202112,CN,301999070,I,중국,아시아.태평양 경제협력체,미꾸라지,수입,643363.0,2593251
31520,202112,JP,302896000,I,일본,아시아.태평양 경제협력체/경제협력개발기구,돔,수입,52.8,2249
31521,202112,CN,302896000,E,중국,아시아.태평양 경제협력체,돔,수출,1696.0,6100


### 실습 데이터셋 전처리

In [11]:
# sfd의 열이름을 출력합니다.
sfd.columns

Index(['stdYymm', 'nationCode', 'mprcExipitmCode', 'imxprtSeCode', 'nationNm',
       'ecoblNm', 'mprcExipitmNm', 'imxprtSeNm', 'imxprtWt',
       'imxprtDollarAmount'],
      dtype='object')

In [12]:
# sfd에서 선택할 열이름을 변수에 할당합니다.
cols = ['stdYymm', 'nationNm', 'mprcExipitmNm', 'imxprtSeNm', 'imxprtWt', 'imxprtDollarAmount']

In [13]:
# sfd에서 cols만 선택한 결과를 출력합니다.
sfd[cols]

,stdYymm,nationNm,mprcExipitmNm,imxprtSeNm,imxprtWt,imxprtDollarAmount
0,202101,대만,클로렐라 효모,수입,500.00,13456
1,202101,일본,클로렐라 효모,수입,84.00,1326
2,202101,미국,클로렐라 효모,수입,4.84,193
3,202101,캐나다,스리루리나 효모,수입,1505.78,33789
4,202101,일본,스리루리나 효모,수입,665.28,17319
...,...,...,...,...,...,...
31517,202112,독일,미꾸라지,수입,1.00,242
31518,202112,미국,미꾸라지,수출,371.00,6209
31519,202112,중국,미꾸라지,수입,643363.00,2593251
31520,202112,일본,돔,수입,52.80,2249


In [14]:
# 인덱싱 결과를 sfd에 재할당합니다.
sfd = sfd[cols]

In [15]:
# sfd의 열이름을 변경합니다.
sfd.columns = ['month', 'nation', 'mpritm', 'imxprt', 'weight', 'dollar']

In [16]:
# sfd의 열별 자료형을 확인합니다.
sfd.dtypes

month       int64
nation     object
mpritm     object
imxprt     object
weight    float64
dollar      int64
dtype: object

In [17]:
# month의 자료형을 문자열로 변환하고 month에 할당합니다. 
# [힌트] astype() 함수를 사용하세요.
sfd['month'] = sfd['month'].astype(str)

/var/folders/w0/zkdyjrqx66g94n4qfn2t2dl40000gn/T/ipykernel_26776/1227254585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sfd['month'] = sfd['month'].astype(str)


In [18]:
# dollar를 1000으로 나눈 결과를 grand에 할당합니다.
# [참고] 데이터프레임 오른쪽 끝에 새 변수를 추가합니다.
sfd['grand'] = sfd['dollar'] / 1000

/var/folders/w0/zkdyjrqx66g94n4qfn2t2dl40000gn/T/ipykernel_26776/3130233863.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sfd['grand'] = sfd['dollar'] / 1000


In [19]:
# sfd에서 dollar를 삭제한 결과를 출력합니다.
# [힌트] drop() 함수를 사용하세요.
sfd.drop(columns = ['dollar'])

,month,nation,mpritm,imxprt,weight,grand
0,202101,대만,클로렐라 효모,수입,500.00,13.456
1,202101,일본,클로렐라 효모,수입,84.00,1.326
2,202101,미국,클로렐라 효모,수입,4.84,0.193
3,202101,캐나다,스리루리나 효모,수입,1505.78,33.789
4,202101,일본,스리루리나 효모,수입,665.28,17.319
...,...,...,...,...,...,...
31517,202112,독일,미꾸라지,수입,1.00,0.242
31518,202112,미국,미꾸라지,수출,371.00,6.209
31519,202112,중국,미꾸라지,수입,643363.00,2593.251
31520,202112,일본,돔,수입,52.80,2.249


In [20]:
# sfd에서 dollar를 삭제한 결과를 sfd에 재할당합니다.
sfd = sfd.drop(columns = ['dollar'])

In [21]:
# sfd의 수치(정수 또는 실수)형 변수의 기술통계량을 확인합니다.
# [힌트] describe() 함수를 사용하세요.
sfd.describe()

,weight,grand
count,3.152200e+04,31522.000000
mean,2.280180e+05,285.383959
std,4.831261e+06,1395.727239
min,0.000000e+00,0.000000
25%,5.200000e+01,0.854000
50%,7.564800e+02,8.355000
75%,1.516800e+04,79.552250
max,3.071980e+08,40737.080000


In [22]:
# sfd의 범주형 변수의 기술통계량을 확인합니다.
sfd.describe(include = object)

,month,nation,mpritm,imxprt
count,31522,31522,28824,31522
unique,12,165,217,2
top,202104,미국,김,수출
freq,2751,3299,1450,17450


In [23]:
# nation의 빈도수를 확인합니다.
# [힌트] value_counts() 함수를 사용하세요.
sfd['nation'].value_counts()

미국        3299
중국        2886
일본        2586
베트남       1932
캐나다       1349
          ... 
조지아          1
세인트루시아       1
우간다          1
가봉           1
통가           1
Name: nation, Length: 165, dtype: int64

In [24]:
# nation의 상대도수를 확인합니다.
# [힌트] value_counts() 함수에 normalize 매개변수를 추가하세요.
sfd['nation'].value_counts(normalize = True)

미국        0.104657
중국        0.091555
일본        0.082038
베트남       0.061291
캐나다       0.042796
            ...   
조지아       0.000032
세인트루시아    0.000032
우간다       0.000032
가봉        0.000032
통가        0.000032
Name: nation, Length: 165, dtype: float64

### 수산물품목별 수출입금액을 합산하고 무역수지 계산

In [25]:
# 아래 조건을 만족하는 피벗 테이블을 mpr에 할당합니다.
# [힌트] pd.pivot_table() 함수를 사용하세요.
# 데이터(data)는 sfd, 값(values)은 grand, 행이름(index)은 mpritm, 
# 열이름(columns)은 imxprt, 집계함수는 np.sum을 지정하세요.
mpr = pd.pivot_table(
    data = sfd, 
    values = 'grand', 
    index = 'mpritm', 
    columns = 'imxprt', 
    aggfunc = np.sum
)

In [26]:
# mpr의 처음 5행을 출력합니다.
mpr.head()

imxprt,수입,수출
mpritm,,
가공하지 않은 것,1.050,0.041
가공한 것,6.872,1.076
가다랑어줄무늬버니토우,2581.407,120207.758
가다랭이,169.757,4937.192
가리비과,50757.462,430.921


In [27]:
# mpr의 행 개수를 출력합니다.
# [힌트] shape 속성 결과에 0번 인덱스 원소를 선택하세요.
mpr.shape[0]

217

In [28]:
# mpr의 열별 결측값 개수를 확인합니다.
mpr.isna().sum()

imxprt
수입    11
수출    23
dtype: int64

In [29]:
# mpr에 있는 결측값을 0으로 대체하고 mpr에 재할당합니다.
# [힌트] fillna() 함수를 사용하세요.
mpr = mpr.fillna(value = 0)

In [30]:
# 수출에서 수입을 뺀 무역수지를 생성합니다.
mpr['무역수지'] = mpr['수출'] - mpr['수입']

In [31]:
# mpr를 무역수지로 내림차순 정렬하고 mpr에 재할당합니다.
# [힌트] sort_values() 함수를 사용하세요.
mpr = mpr.sort_values(by = ['무역수지'], ascending = False)

In [32]:
# mpr의 처음 10행을 출력합니다.
mpr.head(n = 10)

imxprt,수입,수출,무역수지
mpritm,,,
김,1897.017,692914.925,691017.908
다랑어,16423.863,250867.731,234443.868
가다랑어줄무늬버니토우,2581.407,120207.758,117626.351
황다랑어,6581.331,81277.196,74695.865
삼치,60.283,48632.085,48571.802
생선묵,12528.566,59985.938,47457.372
냉동한 것,7294.560,47682.240,40387.680
전복,11498.031,51780.192,40282.161
이빨고기,7709.321,45160.795,37451.474


In [33]:
# mpr의 마지막 10행을 출력합니다.
mpr.tail(n = 10)

imxprt,수입,수출,무역수지
mpritm,,,
분.조분 및 펠리트,121402.589,2648.745,-118753.844
게,225809.371,65924.962,-159884.409
냉동연육,170071.911,2365.547,-167706.364
천일염,172481.091,778.823,-171702.268
새우류,196128.871,2789.326,-193339.545
쭈꾸미,199286.827,307.404,-198979.423
대게,231239.287,6407.234,-224832.053
명태,411130.903,150444.642,-260686.261
낙지,282730.841,818.847,-281911.994


### 특정 수산물품목에 대한 국가별 수출단가 계산

In [34]:
# 아래 조건을 만족하는 피벗 테이블을 itm에 할당합니다.
# [힌트] pd.pivot_table() 함수를 사용하세요.
# 데이터(data)는 sfd에서 특정 조건을 만족하는 행을 지정하세요.
# 값(values)은 grand와 weight를 리스트로 지정하세요.
# 행이름(index)은 nation, 집계함수는 np.sum을 지정하세요.
itm = pd.pivot_table(
    data = sfd[sfd['mpritm'].eq('김') & sfd['imxprt'].eq('수출')], 
    values = ['grand', 'weight'], 
    index = 'nation', 
    aggfunc = np.sum
)

In [35]:
# itm의 처음 5행을 출력합니다.
itm.head()

,grand,weight
nation,,
가나,0.336,32.000
과테말라,71.627,5388.050
괌,228.435,9863.141
그리스,174.915,6312.100
나이지리아,2.108,129.900


In [36]:
# itm의 행 개수를 출력합니다.
itm.shape[0]

114

In [37]:
# itm의 열별 결측값 개수를 확인합니다.
itm.isna().sum()

grand     0
weight    0
dtype: int64

In [38]:
# itm의 열이름을 변경합니다.
itm.columns = ['금액', '중량']

In [39]:
# 금액을 중량으로 나눈 값에 1000을 곱한 수출단가(달러)를 생성합니다.
itm['수출단가'] = itm['금액'] / itm['중량'] * 1000

In [40]:
# itm을 수출단가로 내림차순 정렬하고 itm에 재할당합니다.
# [힌트] sort_values() 함수를 사용하세요.
itm = itm.sort_values(by = ['수출단가'], ascending = False)

In [41]:
# itm의 처음 10행을 출력합니다.
itm.head(n = 10)

,금액,중량,수출단가
nation,,,
부탄,22.730,121.04,187.789161
니카라과,7.713,75.00,102.840000
파키스탄,35.162,556.92,63.136537
모로코,1.660,29.70,55.892256
루마니아,28.034,550.00,50.970909
방글라데시,14.966,299.72,49.933271
바레인,28.146,631.01,44.604681
이집트,18.297,413.60,44.238395
룩셈부르크,0.371,8.48,43.750000


In [42]:
# itm의 마지막 10행을 출력합니다.
itm.tail(n = 10)

,금액,중량,수출단가
nation,,,
브루나이,65.897,5915.94,11.138889
가나,0.336,32.00,10.500000
마다가스카르,2.931,365.00,8.030137
마카오,3.877,487.00,7.960986
잠비아,0.389,52.60,7.395437
팔라우,13.461,2641.18,5.096586
튀니지,0.432,91.40,4.726477
세네갈,0.154,46.00,3.347826
우간다,0.550,560.00,0.982143


### 국가별 수출입금액을 합산하고 무역수지 계산

In [43]:
# 아래 조건을 만족하는 피벗 테이블을 pvt에 할당합니다.
# [힌트] pd.pivot_table() 함수를 사용하세요.
# 데이터(data)는 sfd, 값(values)은 grand, 행이름(index)은 nation, 
# 열이름(columns)은 imxprt, 집계함수는 np.sum을 지정하세요.
pvt = pd.pivot_table(
    data = sfd, 
    values = 'grand', 
    index = 'nation', 
    columns = 'imxprt', 
    aggfunc = np.sum
)

In [44]:
# pvt의 처음 5행을 출력합니다.
pvt.head()

imxprt,수입,수출
nation,,
가나,1847.987,8088.674
가봉,0.026,NaN
가이아나,17.290,0.036
감비아,2065.081,NaN
과테말라,NaN,131.373


In [45]:
# pvt의 행 개수를 출력합니다.
pvt.shape[0]

165

In [46]:
# pvt의 열별 결측값 개수를 확인합니다.
pvt.isna().sum()

imxprt
수입    31
수출    20
dtype: int64

In [47]:
# pvt에 있는 결측값을 0으로 대체하고 pvt에 재할당합니다.
# [힌트] fillna() 함수를 사용하세요.
pvt = pvt.fillna(value = 0)

In [48]:
# 수출에서 수입을 뺀 무역수지를 생성합니다.
pvt['무역수지'] = pvt['수출'] - pvt['수입']

In [49]:
# pvt를 무역수지로 내림차순 정렬하고 pvt에 재할당합니다.
# [힌트] sort_values() 함수를 사용하세요.
pvt = pvt.sort_values(by = ['무역수지'], ascending = False)

In [50]:
# pvt의 처음 5행을 출력합니다.
pvt.head()

imxprt,수입,수출,무역수지
nation,,,
일본,170243.592,634750.935,464507.343
미국,250113.606,394052.534,143938.928
프랑스,34315.325,66501.431,32186.106
이탈리아,25683.273,52960.489,27277.216
독일,1263.392,27185.831,25922.439


In [51]:
# pvt의 행이름을 출력합니다.
# [힌트] index 속성을 사용하세요.
pvt.index

Index(['일본', '미국', '프랑스', '이탈리아', '독일', '홍콩', '싱가포르', '네덜란드', '필리핀', '미령사모아',
       ...
       '아르헨티나', '대만', '캐나다', '인도', '칠레', '페루', '중국', '노르웨이', '베트남', '러시아'],
      dtype='object', name='nation', length=165)

In [52]:
# pvt의 행이름 네임을 '국가'로 변경합니다.
# [힌트] index.name 속성을 사용하세요.
pvt.index.name = '국가'

In [53]:
# pvt의 열이름을 출력합니다.
# [힌트] columns 속성을 사용하세요.
pvt.columns

Index(['수입', '수출', '무역수지'], dtype='object', name='imxprt')

In [54]:
# pvt의 열이름 네임을 삭제합니다.
# [힌트] columns.name 속성을 사용하세요.
# [참고] None을 할당하면 객체를 삭제합니다.
pvt.columns.name = None

In [55]:
# pvt의 처음 5행을 출력합니다.
pvt.head()

,수입,수출,무역수지
국가,,,
일본,170243.592,634750.935,464507.343
미국,250113.606,394052.534,143938.928
프랑스,34315.325,66501.431,32186.106
이탈리아,25683.273,52960.489,27277.216
독일,1263.392,27185.831,25922.439


In [56]:
# pvt의 행이름을 초기화한 결과를 출력합니다.
# [힌트] reset_index() 함수를 사용하세요.
# [참고] 기존 행이름을 맨 처음 열로 추가합니다.
# [참고] drop = True를 추가하면 기존 행이름을 삭제합니다.
pvt.reset_index()

,국가,수입,수출,무역수지
0,일본,170243.592,634750.935,464507.343
1,미국,250113.606,394052.534,143938.928
2,프랑스,34315.325,66501.431,32186.106
3,이탈리아,25683.273,52960.489,27277.216
4,독일,1263.392,27185.831,25922.439
...,...,...,...,...
160,페루,165384.173,2708.109,-162676.064
161,중국,1300449.248,726488.682,-573960.566
162,노르웨이,575973.581,1884.646,-574088.935
163,베트남,808388.014,177117.491,-631270.523


In [57]:
# pvt의 행이름을 초기화하고 pvt에 재할당합니다.
pvt = pvt.reset_index()

In [58]:
# pvt의 처음 10행을 출력합니다.(무역수지 흑자 국가)
pvt.head(n = 10)

,국가,수입,수출,무역수지
0,일본,170243.592,634750.935,464507.343
1,미국,250113.606,394052.534,143938.928
2,프랑스,34315.325,66501.431,32186.106
3,이탈리아,25683.273,52960.489,27277.216
4,독일,1263.392,27185.831,25922.439
5,홍콩,32389.424,54539.763,22150.339
6,싱가포르,2593.154,21322.274,18729.120
7,네덜란드,5373.894,19642.801,14268.907
8,필리핀,29771.506,37645.066,7873.560
9,미령사모아,811.309,8649.957,7838.648


In [59]:
# pvt의 마지막 10행을 출력합니다.(무역수지 적자 국가)
pvt.tail(n = 10)

,국가,수입,수출,무역수지
155,아르헨티나,69720.069,53.530,-69666.539
156,대만,129664.240,58164.339,-71499.901
157,캐나다,113857.715,41894.865,-71962.850
158,인도,106905.400,686.292,-106219.108
159,칠레,119811.899,1426.241,-118385.658
160,페루,165384.173,2708.109,-162676.064
161,중국,1300449.248,726488.682,-573960.566
162,노르웨이,575973.581,1884.646,-574088.935
163,베트남,808388.014,177117.491,-631270.523
164,러시아,1219825.619,53475.703,-1166349.916


## 데이터 병합

### 실습 데이터셋 안내

- 2021년 전세계 국가별 1인당 국민총소득입니다.
  - 출처: 통계청 KOSIS > 국제통계 > 주제별 통계 > 국민계정 > 1인당 국민총소득<br><br>

- 변수(열)에 대한 간단한 설명입니다.
  - nation: 국가명
  - gdp_2019: 2019년 1인당 국민총소득
  - gdp_2020: 2020년 1인당 국민총소득
  - gdp_2021: 2021년 1인당 국민총소득<br><br>

- 실습 데이터인 csv 파일을 아래 링크로 제공합니다.
  - https://bit.ly/GDP_Per_Capita_2021_UTF8
  - 구분자는 콤마(,)이고 인코딩방식은 UTF-8입니다.

In [60]:
# 온라인으로 공유 중인 csv 파일 주소를 변수에 할당합니다.
url = 'https://bit.ly/GDP_Per_Capita_2021_UTF8'

In [61]:
# 링크에 있는 텍스트를 내려받습니다.(HTTP Requests)
res = requests.get(url = url)

In [62]:
# res에서 바이너리 텍스트의 일부를 출력합니다.
res.content[:200]

b'"nation","gdp_2019","gdp_2020","gdp_2021"\n"\xec\x95\x84\xed\x94\x84\xea\xb0\x80\xeb\x8b\x88\xec\x8a\xa4\xed\x83\x84",520,"500","-"\n"\xec\x95\x84\xeb\xa5\xb4\xeb\xa9\x94\xeb\x8b\x88\xec\x95\x84",4660,"4220","4560"\n"\xec\x95\x84\xec\xa0\x9c\xeb\xa5\xb4\xeb\xb0\x94\xec\x9d\xb4\xec\x9e\x94",4510,"4480","4880"\n"\xeb\xb0\x94\xeb\xa0\x88\xec\x9d\xb8",22230,"19930","-"\n"\xeb\xb0\xa9\xea\xb8\x80\xeb\x9d\xbc\xeb\x8d\xb0\xec\x8b\x9c'

In [63]:
# 바이너리 텍스트의 인코딩 방식을 확인합니다.
chardet.detect(res.content[:200])

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [64]:
# 링크를 읽고 데이터프레임 gdp를 생성합니다.
# [참고] 인코딩 방식이 UTF-8이면 encoding 매개변수를 생략할 수 있습니다.
gdp = pd.read_csv(filepath_or_buffer = url, encoding = 'UTF-8')

In [65]:
# gdp의 정보를 확인합니다.
gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   nation    193 non-null    object
 1   gdp_2019  193 non-null    int64 
 2   gdp_2020  193 non-null    object
 3   gdp_2021  193 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.2+ KB


In [66]:
# gdp의 처음 5행을 출력합니다.
gdp.head()

,nation,gdp_2019,gdp_2020,gdp_2021
0,아프가니스탄,520,500,-
1,아르메니아,4660,4220,4560
2,아제르바이잔,4510,4480,4880
3,바레인,22230,19930,-
4,방글라데시,2250,2340,2620


In [67]:
# gdp의 열이름을 출력합니다.
gdp.columns

Index(['nation', 'gdp_2019', 'gdp_2020', 'gdp_2021'], dtype='object')

In [68]:
# gdp에서 일부 열을 삭제합니다.
# [힌트] drop() 함수를 사용하세요.
gdp = gdp.drop(columns = ['gdp_2019', 'gdp_2020'])

In [69]:
# gdp의 마지막 5행을 출력합니다.
gdp.tail()

,nation,gdp_2021
188,사모아,3860
189,솔로몬제도,2300
190,통가,-
191,투발루,6760
192,바누아투,3140


In [70]:
# gdp_2021의 원소가 '-'이면 결측값(np.nan)으로 변경합니다.
# [힌트] np.where()와 eq() 함수를 사용하세요.
gdp['gdp_2021'] = np.where(gdp['gdp_2021'].eq('-'), np.nan, gdp['gdp_2021'])

In [71]:
# gdp의 열별 결측값 개수를 확인합니다.
# [힌트] isna()와 sum() 함수를 사용하세요.
gdp.isna().sum()

nation       0
gdp_2021    21
dtype: int64

In [72]:
# gdp의 정보를 확인합니다.
gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   nation    193 non-null    object
 1   gdp_2021  172 non-null    object
dtypes: object(2)
memory usage: 3.1+ KB


In [73]:
# gdp의 처음 5행을 출력합니다.
gdp.head()

,nation,gdp_2021
0,아프가니스탄,NaN
1,아르메니아,4560
2,아제르바이잔,4880
3,바레인,NaN
4,방글라데시,2620


In [74]:
# gdp에서 결측값이 있는 행을 삭제합니다.
gdp = gdp.dropna()

In [75]:
# gdp의 행 개수를 출력합니다.
# [힌트] shape 속성 결과에 0번 인덱스 원소를 선택하세요.
gdp.shape[0]

172

In [76]:
# gdp_2021의 자료형을 정수로 변환합니다.
gdp['gdp_2021'] = gdp['gdp_2021'].astype(int)

In [77]:
# gdp의 열별 자료형을 확인합니다.
gdp.dtypes

nation      object
gdp_2021     int64
dtype: object

### 데이터 병합

두 데이터프레임을 좌우로 병합할 때 기준이 되는 열을 외래키라고 합니다.<br>
두 외래키에서 일치하는 원소가 없으면 병합했을 때 행 개수가 0일 수 있습니다.<br>
병합 방법은 내부병합, 외부병합, 왼쪽병합 등이 있습니다.<br>
실습 데이터셋 오른쪽에 새로운 열을 추가할 때 왼쪽병합을 주로 사용합니다.

In [78]:
# 왼쪽 외래키에는 있지만 오른쪽 외래키에 없는 원소를 확인합니다.
# [힌트] set() 함수를 사용하여 시리즈를 집합으로 변환하세요.
# 두 집합으로 차집합을 실행하세요.
set(pvt['국가']) - set(gdp['nation'])

{'괌',
 '그린란드',
 '기타국',
 '네덜란드령 안틸레스',
 '누벨칼레도니',
 '니우에',
 '대만',
 '룩셈부르크',
 '마카오',
 '미령사모아',
 '바레인',
 '베네수엘라',
 '부탄',
 '북마리아나제도',
 '브루나이',
 '사우디아라비아',
 '아랍에미리트',
 '예멘',
 '오만',
 '이란',
 '쿠웨이트',
 '키리바시',
 '통가',
 '팔라우',
 '페로제도',
 '포클랜드제도',
 '푸에르토리코',
 '프랑스령 폴리네시아'}

In [79]:
# pvt와 gdp를 왼쪽병합합니다.
# [힌트] pd.merge() 함수를 사용하세요.
# 왼쪽병합은 how 매개변수에 'left'를 지정합니다.
# 외래키는 on 매개변수에 열이름을 문자열로 지정합니다.
# [참고] 외래키 열이름이 다르면 left_on, right_on 매개변수에 각각 지정해야 합니다.
pd.merge(left = pvt, right = gdp, how = 'left', left_on = '국가', right_on = 'nation')

,국가,수입,수출,무역수지,nation,gdp_2021
0,일본,170243.592,634750.935,464507.343,일본,42620.0
1,미국,250113.606,394052.534,143938.928,미국,70430.0
2,프랑스,34315.325,66501.431,32186.106,프랑스,43880.0
3,이탈리아,25683.273,52960.489,27277.216,이탈리아,35710.0
4,독일,1263.392,27185.831,25922.439,독일,51040.0
...,...,...,...,...,...,...
160,페루,165384.173,2708.109,-162676.064,페루,6520.0
161,중국,1300449.248,726488.682,-573960.566,중국,11890.0
162,노르웨이,575973.581,1884.646,-574088.935,노르웨이,84090.0
163,베트남,808388.014,177117.491,-631270.523,베트남,3560.0


In [80]:
# gdp의 열이름을 변경합니다.
# [힌트] rename() 함수를 사용하세요.
gdp = gdp.rename(columns = {'nation': '국가', 'gdp_2021': '국민소득'})

In [81]:
# pvt와 gdp를 왼쪽병합하고 mgd에 할당합니다.
mgd = pd.merge(left = pvt, right = gdp, how = 'left', on = '국가')

In [82]:
# mgd의 정보를 확인합니다.
mgd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165 entries, 0 to 164
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   국가      165 non-null    object 
 1   수입      165 non-null    float64
 2   수출      165 non-null    float64
 3   무역수지    165 non-null    float64
 4   국민소득    137 non-null    float64
dtypes: float64(4), object(1)
memory usage: 7.7+ KB


In [83]:
# mgd의 처음 5행을 출력합니다.
mgd.head()

,국가,수입,수출,무역수지,국민소득
0,일본,170243.592,634750.935,464507.343,42620.0
1,미국,250113.606,394052.534,143938.928,70430.0
2,프랑스,34315.325,66501.431,32186.106,43880.0
3,이탈리아,25683.273,52960.489,27277.216,35710.0
4,독일,1263.392,27185.831,25922.439,51040.0


### 연속형 변수의 구간화(binning)

연속형 변수를 기준에 따라 범주형으로 변환하는 것을 구간화(binning)라고 합니다.<br>
구간화는 이상치와 비선형 문제를 해결하고, 결측값을 쉽게 처리할 수 있습니다.<br>
범주가 2개면 np.where(), 3개 이상이면 np.seelct() 함수를 사용하는 것이 좋습니다.

In [84]:
# 국민소득의 기술통계량을 확인합니다.
# [힌트] describe() 함수를 사용하세요.
mgd['국민소득'].describe()

count      137.000000
mean     15546.423358
std      20658.408477
min        450.000000
25%       2300.000000
50%       6130.000000
75%      17740.000000
max      90360.000000
Name: 국민소득, dtype: float64

In [85]:
# 국민소득이 23,000 달러 이상이면 '선진국', 6,500 ~ 23000 달러 미만이면 '신흥국', 
# 6,500 달러 미만이면 '개도국'인 원소를 갖는 국가등급 열을 생성합니다.
# [힌트] np.select() 함수를 사용하세요.
# condlist 매개변수에 조건, choicelist 매개변수에 원소를 리스트로 지정합니다.
mgd['국가등급'] = np.select(
    condlist = [mgd['국민소득'].ge(23000),
                mgd['국민소득'].ge(10000),
                mgd['국민소득'].lt(10000),
                mgd['국민소득'].isna()],
    choicelist = ['선진국', '신흥국', '개도국', '없음']
)

In [86]:
# 국가등급의 원소별 빈도수를 확인합니다.
# [힌트] value_counts() 함수를 사용하세요.
mgd['국가등급'].value_counts()

개도국    87
선진국    30
없음     28
신흥국    20
Name: 국가등급, dtype: int64

In [87]:
# 국가등급의 원소별 상대도수를 확인합니다.
# [힌트] value_counts() 함수에 normalize 매개변수를 추가하세요.
mgd['국가등급'].value_counts(normalize = True)

개도국    0.527273
선진국    0.181818
없음     0.169697
신흥국    0.121212
Name: 국가등급, dtype: float64

In [88]:
# 국가등급이 '없음'인 행을 출력합니다.
mgd[mgd['국가등급'].eq('없음')]

,국가,수입,수출,무역수지,국민소득,국가등급
9,미령사모아,811.309,8649.957,7838.648,NaN,없음
23,아랍에미리트,550.097,2275.399,1725.302,NaN,없음
29,괌,0.248,1157.706,1157.458,NaN,없음
34,북마리아나제도,0.000,614.004,614.004,NaN,없음
45,쿠웨이트,0.000,171.606,171.606,NaN,없음
48,브루나이,0.000,123.405,123.405,NaN,없음
52,마카오,19.235,126.330,107.095,NaN,없음
58,기타국,1.853,51.576,49.723,NaN,없음
63,부탄,0.000,30.202,30.202,NaN,없음
68,팔라우,0.405,19.791,19.386,NaN,없음


### 외부 파일로 저장

In [89]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

'/Users/hdsceokevin/Documents/Lectures/PythonAdvanced/basic_review'

In [90]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [91]:
# 전역 변수 목록을 출력합니다.
%whos

Variable   Type         Data/Info
---------------------------------
chardet    module       <module 'chardet' from '/<...>ges/chardet/__init__.py'>
cols       list         n=6
gdp        DataFrame             국가   국민소득\n1    <...>n\n[172 rows x 2 columns]
itm        DataFrame                금액       중량  <...>n\n[114 rows x 3 columns]
joblib     module       <module 'joblib' from '/U<...>ages/joblib/__init__.py'>
mgd        DataFrame           국가           수입   <...>n\n[165 rows x 6 columns]
mpr        DataFrame    imxprt               수입  <...>n\n[217 rows x 3 columns]
np         module       <module 'numpy' from '/Us<...>kages/numpy/__init__.py'>
os         module       <module 'os' from '/Libra<...>10/lib/python3.10/os.py'>
pd         module       <module 'pandas' from '/U<...>ages/pandas/__init__.py'>
pvt        DataFrame           국가           수입   <...>n\n[165 rows x 4 columns]
requests   module       <module 'requests' from '<...>es/requests/__init__.py'>
res        Response     

In [92]:
# 여러 객체를 하나의 압축 파일로 저장합니다.
# [힌트] joblib.dump() 함수를 사용하세요.
joblib.dump(value = [sfd, mgd], filename = 'Seafood_Trade_Prep.z')

['Seafood_Trade_Prep.z']

In [93]:
# 작업 경로에 있는 폴더명과 파일명을 출력합니다.
sorted(os.listdir())

['Seafood_Trade_Prep.z',
 'Used_Cars_Price.csv',
 'Used_Cars_Price.xlsx',
 'Used_Cars_Price.z',
 'Used_Cars_Price_Prep.z']

## End of Document